## Scraping seller's data from indiamart website links 
- info needed 
    1. Nature of business 
    2. GST No.
    3. Annual Turnover
    4. Import Export Code (IEC)

### Industries needed - 
    1. Consumer Electronics & household appliances Furniture
    2. Apparel Clothing & Garment: Ladies Dress & Apparel, Kids, Casual Ethnic & Western Wear, Winter Wear & Accessories, Men Shirts, jeans &clothing, Party wedding western and formal wear, Leather Jackets & Garments, Unisex Clothing, 
    3. Cosmetics & Personal Care
    4. Kitchen Essentials
    5. Gifts & Decoratives
    6. Sports Goods, Toys & Games
    7. Gems Jewelry & Astrology
    8. Fashion Accessories
    9. Bags, Belts & Wallets

### IndiaMart Website : https://dir.indiamart.com/

__*XPATH*__ :

**industry name XPATH** : /html/body/div[2]/article[1]/section[4]/h2/a

**first industry XPATH** : /html/body/div[2]/article[1]/section[1]/div[2]/a[7]

**last industry XPATH** : /html/body/div[2]/article[1]/section[56]/div[2]/a[7]


In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [ ]:
df = pd.DataFrame()
ch_driver = webdriver.Chrome(ChromeDriverManager().install())
# Open up a Chrome browser and navigate to web page.
options = webdriver.ChromeOptions()

## Industries

Will get the **Industries list** from the indiamart website using the above mentinoed XPath in a loop.

In [ ]:
url1 = "https://dir.indiamart.com/"
ch_driver.get(url1)
for i in range(1,56):
    Industries = []
    Industries.append(ch_driver.find_element(By.XPATH, "/html/body/div[2]/article[2]/section["+str(i)+"]/h2/a").text)
    Industries.append(ch_driver.find_element(By.XPATH, "/html/body/div[2]/article[2]/section["+str(i)+"]/div[2]/a[7]").get_attribute("href"))
    df = df.append([Industries], ignore_index=True)
df.columns = ['Industry', 'URL']
df.to_csv('Industries.csv',index=False)

## SubIndustries

Using the above industry **URL's**, will now get the subindustries list along with URL's for the selected Industries.

SubIndustry __*Name*__ XPATH : /html/body/div[9]/div[2]/div[1]/ul1/li1/a, *ul* and *li* tags will be iterating in loop

for the URL we should use .get_attribute("href")

In [ ]:
sub_ind_df = pd.DataFrame()
for i in range(1,55):
    if df["Industry"][i] in ('Apparel & Garments','Electronics & Electrical','Furniture & Supplies','Housewares & Supplies',
                             'Handicrafts & Decoratives','Kitchen Utensils & Appliances','Cosmetics & Personal Care','Home Textile & Furnishing',
                             'Gems, Jewelry & Astrology','Fashion Accessories & Gear','Sports Goods, Toys & Games','Bags, Belts & Wallets','Leather Products',):
        #print(df["Industry"][i],i)
        url=df["URL"][i]
        ch_driver.get(url)
        try:
            for j in range(1,20):
                for k in range(1,4):
                    sub_ind=[]
                    sub_ind.append(df["Industry"][i])
                    sub_ind.append(ch_driver.find_element(By.XPATH, "/html/body/div[9]/div[2]/div[1]/ul["+str(j)+"]/li["+str(k)+"]/a").text)
                    sub_ind.append(ch_driver.find_element(By.XPATH, "/html/body/div[9]/div[2]/div[1]/ul["+str(j)+"]/li["+str(k)+"]/a").get_attribute("href"))
                    sub_ind_df = sub_ind_df.append([sub_ind], ignore_index=True)
        except:
            #print(i,"error continued")
            continue
            
sub_ind_df.columns=["Industry","Sub_Industry","URL"]
#sub_ind_df.to_csv('Sub_industries.csv',index=False) 
#already saved

In [ ]:
sub_ind_df=pd.read_csv('Sub_industries.csv')
sub_ind_df

## Product Types

By visting each selected subIndustries will now get the list of different product types along with URL's

will be limiting the loop to 20 products types for each subindustries.

*XPATH* : /html/body/div[9]/div[3]/div/section[1]/ul/li1/div/div[1]/div[1]/div/a

In [ ]:
ch_driver = webdriver.Chrome(ChromeDriverManager().install())
product_df=pd.DataFrame()
for i in range(0,88):
    #print(df["Industry"][i],i)
    url=sub_ind_df["URL"][i]
    ch_driver.refresh()
    try:
        ch_driver.get(url)
    except:
        print(i)
        ch_driver.refresh()
    try:
        for j in range(1,20):
            product=[]
            product.append(sub_ind_df["Industry"][i])
            product.append(sub_ind_df["Sub_Industry"][i])
            product.append(ch_driver.find_element(By.XPATH, "/html/body/div[9]/div[3]/div/section[1]/ul/li["+str(j)+"]/div/div[1]/div[1]/div/a").text)
            product.append(ch_driver.find_element(By.XPATH, "/html/body/div[9]/div[3]/div/section[1]/ul/li["+str(j)+"]/div/div[1]/div[1]/div/a").get_attribute("href"))
            product_df = product_df.append([product], ignore_index=True)
    except:
        #print(i,"error continued")
        continue
            
#product_df.columns=["Industry","Sub_Industry","Product","URL"]
#product_df.to_csv('Products.csv',index=False)


In [ ]:
#product_df.to_csv('Products.csv',index=False)

In [ ]:
products_df=pd.read_csv('Products.csv')
products_df

In [ ]:
seller_df=pd.read_csv('Seller.csv')
#ch_driver.quit()

## Seller Info : 

will now get the seller details using the below code.

    1. Vist the each products URL, go to the different products listed by different sellers.
    2. get the Location and seller name and then vist the first product available under each seller.
    3. These product sites may have 2 different format
        i. have the Home/About tabs in the seller product page directly.
        ii. Or have the about section at the end of the product page. => it further have 2 different class Id's.
        These above 2 cases are handeled using try except.
    4. Get the all Seller details available in any of the above cases.
    5. Will be saving the data in chunks(i.e, every 20 iterations) to avoid the time loss in worst case, as the code might crash due to time out

In [ ]:
ch_driver = webdriver.Chrome(ChromeDriverManager().install())
for i in range(0,967):
    #print(df["Industry"][i],i)
    url=products_df["URL"][i]
    ch_driver.refresh()
    id=['aboutUs','abt']
    if(i%20==0): #save a backup record
        seller_df=seller_df.drop_duplicates()
        #seller_df.to_csv('Seller.csv',index=False)
    try:
        ch_driver.get(url.split(".html")[0]+"-all.html")
    except:
        print(i)
        ch_driver.refresh()
    #ch_driver.refresh()
    n=int(ch_driver.find_element(By.XPATH, "//div[@id='content']/div/div[2]/span").text.lstrip('(').split(' ')[0])
    if n>20:
        n=20
    for j in range(0,n):
        try:
            seller={"Industry":"","Sub_Industry":"","Seller_Name":"","Location":"","Year of Establishment":"","IndiaMART Member Since":"","Total Number of Employees":"","Legal Status of Firm":"","Nature of Business":"","Annual Turnover":"","GST Number":"","Import Export Code (IEC)":""}
            ch_driver.get(url.split(".html")[0]+"-all.html")
            seller["Industry"]=products_df["Industry"][i]
            seller["Sub_Industry"]=products_df["Sub_Industry"][i]
            seller["Seller_Name"]=ch_driver.find_element(By.XPATH, "//div[@id='LST"+str(j)+"']/div[1]/div[1]/div[1]/span[1]/a").text #seller name
            seller["Location"]=ch_driver.find_element(By.XPATH, "//div[@id='LST"+str(j)+"']/div[1]/div[1]/div[2]").text #seller location
            ch_driver.get(ch_driver.find_element(By.XPATH, "//div[@id='gu"+str(j)+"_0']/div[1]/a").get_attribute("href"))
            try:
                ch_driver.find_element(By.XPATH, "//div[@id='"+str(id[0])+"']")
                k=0
            except:
                try:
                    ch_driver.find_element(By.XPATH, "//div[@id='"+str(id[1])+"']")
                    k=1
                except:
                    ch_driver.get(url.split(".html")[0]+"-all.html")
                    seller={"Industry":"","Sub_Industry":"","Seller_Name":"","Location":"","Year of Establishment":"","IndiaMART Member Since":"","Total Number of Employees":"","Legal Status of Firm":"","Nature of Business":"","Annual Turnover":"","GST Number":"","Import Export Code (IEC)":""}
                    seller["Industry"]=products_df["Industry"][i]
                    seller["Sub_Industry"]=products_df["Sub_Industry"][i]
                    seller["Seller_Name"]=ch_driver.find_element(By.XPATH, "//div[@id='LST"+str(j)+"']/div[1]/div[1]/div[1]/span[1]/a").text #seller name
                    seller["Location"]=ch_driver.find_element(By.XPATH, "//div[@id='LST"+str(j)+"']/div[1]/div[1]/div[2]").text #seller location
                    ch_driver.get(ch_driver.find_element(By.XPATH, "//div[@id='gu"+str(j)+"_0']/div[1]/a").get_attribute("href").rsplit('/',1)[0])
                    for s in range (1,9):
                        try:
                            seller[ch_driver.find_element(By.XPATH, "/html/body/main/div[2]/div[3]/div[1]/div[3]/div[3]/div["+str(s)+"]/div[2]/p").text]=ch_driver.find_element(By.XPATH, "/html/body/main/div[2]/div[3]/div[1]/div[3]/div[3]/div["+str(s)+"]/div[2]/span").text
                        except:
                            pass
                    #print("case 2 : ",seller["Seller_Name"],seller["Year of Establishment"])
                    temp=pd.DataFrame([seller])
                    seller_df = pd.concat([seller_df,temp], ignore_index=True)
                    continue
            #print("In case 1")
            for s in range(1,9):
                try:
                    seller[ch_driver.find_element(By.XPATH, "//div[@id='"+str(id[k])+"']/div[3]/div["+str(s)+"]/span[1]").text]=ch_driver.find_element(By.XPATH, "//div[@id='"+str(id[k])+"']/div[3]/div["+str(s)+"]/span[2]").text
                except:
                    pass
            #print("case 1 : ",seller["Seller_Name"],seller["Year of Establishment"])
            temp=pd.DataFrame([seller])
            seller_df = pd.concat([seller_df,temp], ignore_index=True)
            continue
        except:
            #print("error in")
            if(j==n-1):
                break
            continue

seller_df=seller_df.drop_duplicates()
#seller_df.columns=["Industry","Sub_Industry","Seller Name","Location","Year of Establishment","Nature of Business","Annual Turnover","GST","IEC"]
#seller_df.to_csv('Seller.csv',index=False)


In [ ]:
len(seller_df)
#seller_df.to_csv('Seller.csv',index=False)

### Remove Duplicates and save the seller data

Combine any columns that are same, for example below GST & GST Number columns are same.

Then remove duplicates using **df.drop_duplicates()**

In [ ]:
unique_seller_df=seller_df[["Seller_Name", "Location", "Year of Establishment", "IndiaMART Member Since", "Total Number of Employees", "Legal Status of Firm", "Nature of Business", "Annual Turnover", "GST Number", "Import Export Code (IEC)", "Number of Employees", "GST", "Exports to"]]

In [ ]:
unique_seller_df["GST Number"]=unique_seller_df["GST Number"].fillna(unique_seller_df["GST"])

In [ ]:
unique_seller_df[["GST Number","GST"]]
del unique_seller_df["GST"]

In [ ]:
unique_seller_df=unique_seller_df.drop_duplicates()
unique_seller_df.to_csv('Unique_Sellers.csv',index=False)